# Análisis de Datos para Detección de Accidentes de Coche

Este notebook analiza el dataset de imágenes para la detección de accidentes de coche. Exploraremos las características del dataset, realizaremos visualizaciones y prepararemos los datos para el entrenamiento.

In [5]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install Pillow
!pip install tensorflow

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Configurar el estilo de las gráficas
plt.style.use('seaborn')
sns.set_palette('husl')

ModuleNotFoundError: No module named 'numpy'

## 1. Descripción de los Datos

El dataset está organizado en dos carpetas principales:
- `RoadAccidents`: Imágenes de accidentes de coche
- `NormalVideos`: Imágenes de tráfico normal sin accidentes

Vamos a analizar la estructura y distribución de los datos:

In [7]:
def analyze_dataset(data_dir='data'):
    classes = ['NormalVideos', 'RoadAccidents']
    stats = {}
    
    for class_name in classes:
        class_path = os.path.join(data_dir, class_name)
        if os.path.exists(class_path):
            # Contar imágenes
            images = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            
            # Analizar una muestra de imágenes para dimensiones
            sample_img = Image.open(os.path.join(class_path, images[0]))
            
            stats[class_name] = {
                'num_images': len(images),
                'sample_dimensions': sample_img.size,
                'sample_mode': sample_img.mode
            }
    
    return stats

dataset_stats = analyze_dataset()
print("\nEstadísticas del Dataset:")
for class_name, stats in dataset_stats.items():
    print(f"\n{class_name}:")
    print(f"- Número de imágenes: {stats['num_images']}")
    print(f"- Dimensiones de muestra: {stats['sample_dimensions']}")
    print(f"- Modo de color: {stats['sample_mode']}")


Estadísticas del Dataset:


## 2. Visualización de la Distribución de Clases

In [8]:
def plot_class_distribution(stats):
    classes = list(stats.keys())
    counts = [stats[c]['num_images'] for c in classes]
    
    plt.figure(figsize=(10, 6))
    sns.barplot(x=classes, y=counts)
    plt.title('Distribución de Clases en el Dataset')
    plt.xlabel('Clase')
    plt.ylabel('Número de Imágenes')
    
    # Añadir porcentajes
    total = sum(counts)
    for i, count in enumerate(counts):
        percentage = count/total * 100
        plt.text(i, count, f'{percentage:.1f}%', ha='center', va='bottom')
    
    plt.show()

plot_class_distribution(dataset_stats)

NameError: name 'plt' is not defined

## 3. Visualización de Muestras de Imágenes

In [ ]:
def plot_sample_images(data_dir='data', samples_per_class=5):
    classes = ['NormalVideos', 'RoadAccidents']
    plt.figure(figsize=(15, 6))
    
    for i, class_name in enumerate(classes):
        class_path = os.path.join(data_dir, class_name)
        images = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        samples = np.random.choice(images, samples_per_class, replace=False)
        
        for j, img_name in enumerate(samples):
            plt.subplot(2, samples_per_class, i*samples_per_class + j + 1)
            img = Image.open(os.path.join(class_path, img_name))
            plt.imshow(img)
            plt.axis('off')
            if j == 0:
                plt.title(class_name)
    
    plt.tight_layout()
    plt.show()

plot_sample_images()

## 4. Análisis de Aumentación de Datos

Visualizaremos cómo funcionan las técnicas de aumentación de datos que usamos en el entrenamiento:

In [ ]:
def plot_augmented_samples(data_dir='data', n_samples=5):
    # Configurar el generador de datos con los mismos parámetros del entrenamiento
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        brightness_range=[0.9, 1.1],
        fill_mode='nearest'
    )
    
    # Tomar una imagen de muestra
    sample_path = os.path.join(data_dir, 'RoadAccidents')
    sample_img = np.array(Image.open(os.path.join(sample_path, os.listdir(sample_path)[0])))
    sample_img = sample_img.reshape((1,) + sample_img.shape)
    
    # Generar y mostrar imágenes aumentadas
    plt.figure(figsize=(15, 3))
    plt.subplot(1, n_samples+1, 1)
    plt.imshow(sample_img[0])
    plt.title('Original')
    plt.axis('off')
    
    for i, batch in enumerate(datagen.flow(sample_img, batch_size=1)):
        plt.subplot(1, n_samples+1, i+2)
        plt.imshow(batch[0])
        plt.title(f'Aumentada {i+1}')
        plt.axis('off')
        if i >= n_samples-1:
            break
            
    plt.tight_layout()
    plt.show()

plot_augmented_samples()

## 5. Preparación de Datos para el Entrenamiento

Mostramos cómo se preparan los datos para el entrenamiento usando los generadores de datos:

In [ ]:
def setup_data_generators(data_dir='data', img_size=128, batch_size=32, validation_split=0.2):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        brightness_range=[0.9, 1.1],
        validation_split=validation_split
    )

    valid_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=validation_split
    )

    train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='binary',
        classes=['NormalVideos', 'RoadAccidents'],
        subset='training'
    )

    validation_generator = valid_datagen.flow_from_directory(
        data_dir,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='binary',
        classes=['NormalVideos', 'RoadAccidents'],
        subset='validation'
    )

    return train_generator, validation_generator

train_generator, validation_generator = setup_data_generators()

print("\nConfiguración de los generadores de datos:")
print(f"Pasos por época (entrenamiento): {len(train_generator)}")
print(f"Pasos por época (validación): {len(validation_generator)}")
print(f"Tamaño de batch: {train_generator.batch_size}")
print(f"Tamaño de imagen: {train_generator.target_size}")

## 6. Cálculo de Class Weights

Para manejar el desbalance de clases, calculamos los pesos de cada clase:

In [ ]:
def calculate_class_weights(generator):
    total = len(generator.classes)
    class_counts = np.bincount(generator.classes)
    class_weights = {
        0: total / (2 * class_counts[0]),
        1: total / (2 * class_counts[1])
    }
    
    plt.figure(figsize=(8, 5))
    plt.bar(['Normal', 'Accidente'], [class_weights[0], class_weights[1]])
    plt.title('Pesos de las Clases')
    plt.ylabel('Peso')
    plt.show()
    
    return class_weights

class_weights = calculate_class_weights(train_generator)
print("\nPesos calculados para cada clase:")
print(f"Normal: {class_weights[0]:.3f}")
print(f"Accidente: {class_weights[1]:.3f}")